### 序列可迭代的原因
需要迭代对象x时,python会自动调用iter(x).内置函数iter执行以下操作
1. 检查对象是否实现__iter__方法,如果实现就调用它,获取一个迭代对象
2. 如果没有实现__iter__方法,但实现了__getitem__方法,那么iter()创建一个迭代器,尝试从索引为0的获取项
3. 如果尝试失败,则python抛出TypeError

> 如果对象指实现了__getitem__没实现__iter__,将无法使用isinstance(abc.Iterable,obj)来判断是不是可迭代对象.
Iterable实现了__subclasshook__

In [15]:
class Spam:
    def __getitem__(self,index):
        print(index)
        raise IndexError()

i1=iter(Spam())
arr1=list(i1)
print(arr1)

class IterSpam:
    def __iter__(self):
        yield 1

i2=iter(IterSpam())

from collections.abc import Iterable
print(isinstance(Spam(),Iterable),isinstance(IterSpam(),Iterable))



0
[]
False True


### 使用iter处理可调用对象
调用iter()传入两个参数,第一是可调用对象重复调用产生值,第二个哨符,即一种标记值如果可调用对象返回哨符,则迭代器抛出StopIteration

In [19]:
import random
def make():
    return random.randint(1,6)

e6=iter(make,6)
while (i:=next(e6)) !=6:
    print(i) 
     

3
1
4
1


StopIteration: 

### 可迭代对象与迭代器

可迭代对象:使用内置函数iter可以获取迭代器的对象,如果对象实现了能返回迭代器的__iter__方法,那么对象就是可迭代的.实现了__getitem__方法,而且接受从0开始索引,这种对象也是可迭代的

> python从迭代对象中获取迭代器

python标准中迭代器接口有以下两个方法:
- `__next__`:返回序列中的下一项,如果没有项则抛出StopIteration
- `__iter__`:返回self,以便在预期可迭代对象的地方使用迭代器

**迭代器是一次性的,如果在想迭代必须重构迭代器**

In [3]:
# 经典迭代器

class MyIter:
    def __init__(self,words):
        self.words=words

    def __iter__(self):
        return IterInstance(self.words)

class IterInstance:
    def __init__(self,words):
        self.words=words
        self.index=0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        i=self.index
        if self.index==len(self.words):
            raise StopIteration()
        self.index+=1
        return self.words[i]

words=['a','b','c']

m=MyIter(words)
mi=iter(m)
for i in mi:
    print(i)

a
b
c


### 生成器
只要python函数的主体中有yield关键字,该函数就是生成器函数.调用生成器函数,返回一个生成器对象.yield可以使用多个,生成器函数创建一个生成器对象,包装生成器函数的主体,把生成器对象传给next()函数时,生成器函数提前执行函数主体的下一个yield,返回产出的值并在函数主体的当前位置暂停,最终函数主体返回时,python创建外层生成器对象抛出StopIteration异常

迭代器和生成器对比:
- 迭代器:泛指实现`__next__`方法的对象,可通过for循环方式,或者使用next()来驱动迭代器
- 生成器:由python编译器构建的迭代器,为了创建生成器不必实现`__next__`方法,而使用yield关键字得到生成器函数

In [12]:
def gen():
    print('start..')
    yield 'A'
    print('gen A ')
    yield 'B'
    print('gen B ')
    print('start..')

for c in gen():
    print(c)

print(type(gen()))
arr=(i for i in range(1))
print(type(arr))

class Arithmetic:
    def __init__(self,begin,end,step):
        self.begin=begin
        self.end=end
        self.step=step
    
    def __iter__(self):
        i=self.begin
        j=self.end
        k=self.step
        while i < j:
            yield i
            i+=k

a=Arithmetic(0,10,1)

for i in a:
    print(i) 

import itertools
# itertools.count(begin,step) 会不停生成
for i in itertools.takewhile(lambda val :val < 10,itertools.count(step=1)):
    # print(i)
    pass

        

start..
A
gen A 
B
gen B 
start..
<class 'generator'>
<class 'generator'>
0
1
2
3
4
5
6
7
8
9
9


### itertools标准库的生成器函数

### yield from从子生产器中产出
yiled from 把一个生成器的工作委托给一个子生成器

In [ ]:
def gen():
    yield 'start'
    yield from sub_gen()
    yield 'end'

def sub_gen():
    for i in range(10):
        yield i

for v in gen():
    print(v)


def chain(*iterables):
    for iter in iterables:
        yield from iter
a=[0,1,2,3]
b=[6,7,8]

for i in chain(a,b):
    print(i)


### 泛化可迭代类型
可迭代对象的泛化参数collections.abc.Iterable

> Iterator类型用于yield关键字和__next__方法的迭代类,这个很少在用,还有一个collections.abc.Generator

In [17]:
from collections.abc import Iterable

from typing import TypeAlias

From:TypeAlias=[str,str]

def zip(elements:[str])->Iterable[From]:
    pass




1

### 协程
协程就是生成器函数,通过主体中含有yield关键字函数创建.协程对象就是生成器对象.在使用协程时必须先出示,可以使用next或gen.send(None)这个过程叫'预激携程'.激活后协程停在yield处,等待发送值到来然后在赋值给yield等号的变量

停止协程直接调用close方法,调用close方法后的协程,再调用send就会抛出StopInteration.close方法在暂停的yield处抛出GeneratorExit,如果协程函数没有处理则该异常终止协程,而GeneratorExit被包装协程的生成器对象捕获

> collections.abc.Generator[YieldType,SendType,ReturnType]

In [10]:
def avg():
    val=0.
    count=0
    result=0.
    while True:
        receive=yield result # 如果这里有接收值不能调用next来驱动协程[这里]
        print(receive,val,count)
        val+=receive
        count+=1
        result=val/count

a=avg() 
# 预启动 执行到第一个yield
next(a)       

i1=a.send(2)
i2=a.send(2)
i3=a.send(5)
print(i1,i2,i3)

# 带有返回值
from typing import NamedTuple
class Result(NamedTuple):
    count:int
    average:float

class Sentinel:
    def __repr__(self) -> str:
        return f'<Sentinel>'

from typing import Union
from collections.abc import Generator
STOP=Sentinel()

SendType=Union[float,Sentinel]

# 带有返回值的协程,如果通过close来关闭协程,它直接在yield出抛出GeneratorExit异常不执行result
def avg2()->Generator[None,SendType,Result]:
    val=0.
    count=0
    result=0.
    while True:
        receive=yield 
        if isinstance(receive,Sentinel):
            break
        val+=receive
        count+=1
        result=val/count
    return Result(result,count) 
a2=avg2()
a2.send(None)

a2.send(4)
a2.send(2)
try:
    a2.send(STOP) # 跳出异常协程包装器将抛出StopIteration异常
except StopIteration as ex:
    print(ex.value) # 通过获取StopIteration的value值来获结果


2 0.0 0
2 2.0 1
5 4.0 2
2.0 2.0 3.0
Result(count=3.0, average=2)
